## Korfali et al. 2010
### Mouse leukocytes

In [1]:
import pandas as pd

In [3]:
# Excel import
df_Korfali_2010 = pd.read_excel('./SourceData/Papers/Korfali_2010/rk1-139_fromTableS5.xlsx')

# convert to df
df_Korfali_2010= pd.DataFrame(df_Korfali_2010.iloc[1:, 1])
df_Korfali_2010.columns = ['Gene_name']
df_Korfali_2010 = df_Korfali_2010.reset_index(drop=True)

In [5]:
# Manual inspection and correction
df_Korfali_2010.iloc[15, 0] = 'DHRS7'
df_Korfali_2010.iloc[39, 0] = 'NOC2L'
df_Korfali_2010.iloc[58, 0] = 'METTL7A'
df_Korfali_2010.iloc[104, 0] = 'TMEM189'

In [14]:
# uniprot API URL
WEBSITE_API = "https://rest.uniprot.org/uniprotkb/"

# organism id
organism_id_list = {'Homo sapiens': '9606', 'Mus musculus': '10090'}
organism_id = organism_id_list['Homo sapiens']

# find Uniprot Entry and gene names for each gene
for i in range(len(df_Korfali_2010)):
    
    # gene name
    gene = df_Korfali_2010.iloc[i, 0]

    try:
        # get response with gene name the query
        r = get_url(f'{WEBSITE_API}/search?query=gene:{gene}+AND+organism_id:{organism_id}&fields=accession,gene_names')
        result = r.json()['results'][0]
        
        ## get human or mouse entry
        entry_converted = result['primaryAccession']
        ## get gene name and check the names match between human's or mouse's and the given organism's
        gene_obtained = result['genes'][0]['geneName']['value']
    
    except:
        entry_converted = 'Not_found'
        gene_obtained = 'Not_found'
    
    # Put the obtained Entry and gene name
    df_Korfali_2010.loc[i, 'Entry_Korfali_2010'] = entry_converted
    df_Korfali_2010.loc[i, 'Gene_name_obtained'] = gene_obtained
    
    if i % 40 == 0: print(i, entry_converted, gene_obtained)
    
    # take a break and go next
    sleep(1)

0 Q9UH99 SUN2
40 Q3LXA3 TKFC
80 Q6NXT6 TAPT1
120 Q9Y2W6 TDRKH


In [28]:
# # Manually check the result
# df_Korfali_2010.head(n=50)
# # df_Korfali_2010.tail(n=50)

In [15]:
df_Korfali_2010 = df_Korfali_2010.drop(['Gene_name'], axis=1)

In [16]:
df_Korfali_2010.to_csv('./Output/Korfali_2010.csv', index=False)